In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv("datasets/Online Retail.csv")

In [4]:
display(data)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [24]:
data=data[["InvoiceNo","StockCode","Quantity","InvoiceDate","UnitPrice","CustomerID"]].dropna()

data.insert(5, "Total", data["Quantity"]*data["UnitPrice"], True)
data=data[data["Quantity"]>=0]

In [29]:

customers=data["CustomerID"].unique()

customer=data[data["CustomerID"]==customers[0]]


f=open(f'output/{customers[0]}.csv', "w")
customer.to_csv(path_or_buf=f,index=False,line_terminator='\n')

f.close()
    

5391.210000000001

In [40]:
totals={"CustomerID":[],"Total":[]}
for c in customers:
    customer=data[data["CustomerID"]==c]
    total=customer["Total"].sum()
    totals["CustomerID"].append(c)
    totals["Total"].append(total)

totals=pd.DataFrame(totals)
f=open(f'output/monetary_vars.csv', "w")
totals.to_csv(path_or_buf=f,index=False,line_terminator='\n')
f.close()

In [46]:
num_invoices={"CustomerID":[],"# of invoices":[]}

for c in customers:
    customer=data[data["CustomerID"]==c]
    num=len(customer["InvoiceNo"].unique())
    num_invoices["CustomerID"].append(c)
    num_invoices["# of invoices"].append(num)
num_invoices=pd.DataFrame(num_invoices)
f=open(f'output/frequency.csv', "w")
num_invoices.to_csv(path_or_buf=f,index=False,line_terminator='\n')
f.close()